# VacationPy


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("cities.csv", encoding = "utf-8")
weather_data.head()

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC)
0,Vila Franca do Campo,37.7167,-25.4333,55.4,62,40,8.05,PT,1612485960
1,Ushuaia,-54.8000,-68.3000,50.0,76,20,34.52,AR,1612485276
2,Cape Town,-33.9258,18.4232,68.0,82,0,10.36,ZA,1612485554
3,Busselton,-33.6500,115.3333,64.0,59,4,1.99,AU,1612485527
4,Hilo,19.7297,-155.0900,73.4,73,75,17.27,US,1612485961


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
city_locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity (%)"].astype(float)

In [5]:
fig = gmaps.figure(zoom_level=1.9, center=(25,0))

heat_layer = gmaps.heatmap_layer(city_locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# reduce dataframe to include only cities matching a certain criteria
# max temp>=75, <85
# wind speed<5mph
# cloudiness<= 5

reduced_wd = weather_data.loc[(weather_data["Max Temp. (F)"]>=75) &
                             (weather_data["Max Temp. (F)"]<85) &
                             (weather_data["Wind Speed (mph)"]<5) &
                              (weather_data["Cloudiness (%)"]<=15)]
reduced_wd

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC)
199,Ballitoville,-29.5390,31.2144,75.00,83,0,2.30,ZA,1612486006
383,Coyhaique,-45.5752,-72.0662,77.00,33,0,3.44,CL,1612486048
388,Cândido Mendes,-1.4467,-45.7167,76.57,90,1,3.69,BR,1612486049
394,Coahuayana Viejo,18.7333,-103.6833,78.80,69,1,2.75,MX,1612486050
470,Suphan Buri,14.4742,100.1222,75.00,63,0,4.36,TH,1612486068
502,Yaan,7.3833,8.5667,76.50,18,0,1.86,NG,1612486075
511,Cabedelo,-6.9811,-34.8339,78.80,83,0,4.61,BR,1612486077
570,Lagos,6.5833,3.7500,79.74,86,0,2.91,NG,1612485889


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = reduced_wd.copy()
hotel_df["Hotel Name"] = ""
hotel_df.dropna()
hotel_df

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC),Hotel Name
199,Ballitoville,-29.5390,31.2144,75.00,83,0,2.30,ZA,1612486006,
383,Coyhaique,-45.5752,-72.0662,77.00,33,0,3.44,CL,1612486048,
388,Cândido Mendes,-1.4467,-45.7167,76.57,90,1,3.69,BR,1612486049,
394,Coahuayana Viejo,18.7333,-103.6833,78.80,69,1,2.75,MX,1612486050,
470,Suphan Buri,14.4742,100.1222,75.00,63,0,4.36,TH,1612486068,
502,Yaan,7.3833,8.5667,76.50,18,0,1.86,NG,1612486075,
511,Cabedelo,-6.9811,-34.8339,78.80,83,0,4.61,BR,1612486077,
570,Lagos,6.5833,3.7500,79.74,86,0,2.91,NG,1612485889,


In [8]:
# https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
params = {"radius": 5000,
         "types": "lodging",
          "key":g_key}
          
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
#     print(hotel_name.url)
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.


,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC),Hotel Name
199,Ballitoville,-29.5390,31.2144,75.00,83,0,2.30,ZA,1612486006,Hotel Fairmont Zimbali Resort
383,Coyhaique,-45.5752,-72.0662,77.00,33,0,3.44,CL,1612486048,Diego De Almagro Coyhaique
388,Cândido Mendes,-1.4467,-45.7167,76.57,90,1,3.69,BR,1612486049,Quebrada Do Mlk PJL
394,Coahuayana Viejo,18.7333,-103.6833,78.80,69,1,2.75,MX,1612486050,Hotel Los Arcos
470,Suphan Buri,14.4742,100.1222,75.00,63,0,4.36,TH,1612486068,Vasidtee City Hotel
502,Yaan,7.3833,8.5667,76.50,18,0,1.86,NG,1612486075,
511,Cabedelo,-6.9811,-34.8339,78.80,83,0,4.61,BR,1612486077,Hotel Pigalle
570,Lagos,6.5833,3.7500,79.74,86,0,2.91,NG,1612485889,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))